<a href="https://colab.research.google.com/github/lucasbenazzi/FCC-MachineLearningExercises/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-01-18 21:37:02--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  45.7MB/s    in 0.5s    

2022-01-18 21:37:03 (45.7 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code

In [ ]:
#removing users with less than 200 ratings
#for a user to have less than 200 ratings, it has to appear less then 200 times on df_ratings
user_rating_count = (df_ratings['user'].value_counts() >= 200)
user_rating_list = user_rating_count[user_rating_count==True].index

user_rating_filter = df_ratings['user'].isin(user_rating_list)

#remove books with less than 100 ratings
#for a book to have less then 100 ratings, it has to appear less then 100 times on df_ratings
book_rating_count = (df_ratings['isbn'].value_counts() >= 100)
book_rating_list = book_rating_count[book_rating_count==True].index

book_rating_filter = df_ratings['isbn'].isin(book_rating_list)

In [ ]:
#create "new_df_ratings" with both filters
new_df_ratings = df_ratings[user_rating_filter & book_rating_filter]


#create "new_df_books" based on the books of the "new_df_ratings" variable
#this step is not necessary, is just there so there is a "new_df_book" variable only with the isbn values on the "new_df_ratings" variable
new_isbn_list = new_df_ratings['isbn'].values
new_df_books_filter = df_books['isbn'].isin(new_isbn_list)

new_df_books = df_books[new_df_books_filter]

In [ ]:
#prepare data to train by pivoting the data
df_ratings_pivot = new_df_ratings.pivot(
    index='isbn', columns='user', values='rating').fillna(0)

In [ ]:
#training the model
#since there are many dimentions, using euclidean distance wouldn't be accurate.
#to resolve this issue, it's used "cosine" metric
nbrs = NearestNeighbors(algorithm='auto', metric = 'cosine').fit(df_ratings_pivot)

In [ ]:
# creating function to get the title of a book based on the id of the "df_ratings_pivot" variable
def get_name_from_id(id):
  try:
    isbn = df_ratings_pivot.iloc[id].name
    return df_books.loc[df_books['isbn'] == isbn, 'title'].values[0]
  # some books are on df_rating variable, but not on df_books, on those cases, there is no title match for the isbn
  # since dealing with this issue is not a part of this exercise, leave the titles as blank
  except:
    return ''

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # get list of isbn codes based on the name, as some names have multiple isbn codes
  try:
    isbn_list = df_books.loc[df_books['title'] == book, 'isbn'].values
  
  # since it's not the focus of this project, this script doesn't work if the given string isn't a perfect match with a book title
  # for a future project, develop a system to search for similar titles
  except:
    print("No title with given name")
    return False

  # create a list for the cases where a given title has multiple isbn
  final_results = []
  for isbn in isbn_list:
    #n_neighbors is set to 6, since the "kneighbors" function returns the search variable itself
    distances, indices = nbrs.kneighbors([df_ratings_pivot.loc[isbn]],n_neighbors = 6)

    #creating a pandas DataFrame as opposed to a list from the results, as a easy way to order the recommended books by distance
    df_recommendations = pd.DataFrame(
        list(zip(indices[0], distances[0])),
        columns = ['id','distance']
        ).set_index('id').sort_values('distance',ascending = False)
    #drop value with distance == 0
    df_recommendations = df_recommendations[df_recommendations.distance != 0]
    
    # create a list with the specified format for the exercise
    recommendation_list = []
    for id, distance in df_recommendations.iterrows():
      result = [get_name_from_id(id), distance[0]]
      recommendation_list.append(result)
    final_results.append([book, recommendation_list])

  # if there are multiple isbn for a given title, return all results
  if len(final_results) > 1:
    return final_results
  # if there is only one isbn, return the first index of the "final_results" to match the exercise desired output
  else:
    return final_results[0]

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
You passed the challenge! 🎉🎉🎉🎉🎉


##Sources

1. "Prototyping a Recommender System Step by Step Part 1: KNN Item-Based Collaborative Filtering" towardsdatascience, https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea
2. "1.6. Nearest Neighbors" scikit-learn, https://scikit-learn.org/stable/modules/neighbors.html
3. "sklearn.neighbors.NearestNeighbors" scikit-learn, https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
4. "pandas.DataFrame.pivot" pandas, https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html
5. "pandas.DataFrame.sort_values" pandas, https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html

### Extra links
  The answer on the stackoverflow post from "Abhinav Gupta" and "legoscia" helped with a shorter sintaxe: https://stackoverflow.com/questions/30522724/take-multiple-lists-into-dataframe

